In [2]:
# image features를 .pkl 파일에 저장한다. (test)




In [1]:
import os

In [2]:
data_dir="/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/test/"
print(os.listdir(data_dir))

for sub_dir in os.listdir(data_dir):
    dir = data_dir+sub_dir
    print(dir)

['785', '784']
/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/test/785
/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/test/784


In [4]:

# 1. 파일 경로 불러오기


dir_num=785

data_dir="/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/test/"+ str(dir_num)+"/"

all_files = [os.path.join(data_dir, path) for path in os.listdir(data_dir) if path[0] != "." and path.endswith(".jpg")] # 이미지 파일 경로

print(dir_num)
print(f"폴더 내 파일 개수: {len(all_files)}")
all_files=sorted(all_files)
print(all_files)



785
폴더 내 파일 개수: 727
['/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/test/785/785_frame_0.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/test/785/785_frame_1.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/test/785/785_frame_10.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/test/785/785_frame_100.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/test/785/785_frame_101.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/test/785/785_frame_102.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/test/785/785_frame_103.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/test/785/785_frame_104.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/test/785/785_frame_105.jpg', '/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterl

In [5]:
from typing import Tuple
class CreateArg():
    def __init__(self):
        # Dataset options
        self.dataset_name = 'waterloo'
        self.delim = '\t'
        self.loader_num_workers = 4
        self.obs_len = 8
        self.pred_len = 8
        self.skip = 1
        # Optimization
        self.batch_size = 4                 #### batch_size test !!!
        self.num_iterations = 1000 # 10000
        self.num_epochs = 50 # 200                     
        # Model Options
        self.embedding_dim = 64
        self.num_layers = 1
        self.dropout = 0.0
        self.batch_norm = 0 
        self.mlp_dim = 1024
        # self.default_backbone= 'resnet18'
                                    
        # Generator Options
        self.encoder_h_dim_g = 64
        self.decoder_h_dim_g = 128
        self.noise_dim : Tuple[int] = (0, 0) # default=None # type=int_tuple
        self.noise_type = 'gaussian'
        self.noise_mix_type = 'ped'
        self.clipping_threshold_g = 0 # type=float
        self.g_learning_rate = 5e-4 # type=float 
        self.g_steps = 1

        # Pooling Options
        self.pooling_type = 'pool_net' 
        self.pool_every_timestep = 1 # type=bool_flag

        # Pool Net Option
        self.bottleneck_dim = 1024 # type=int

        # Social Pooling Options
        self.neighborhood_size = 1024 # type=float
        self.grid_size = 8 # type=int

        # Discriminator Options
        self.d_type = 'local' # type=str
        self.encoder_h_dim_d = 64 # type=int
        self.d_learning_rate = 5e-4 # type=float
        self.d_steps = 2 # type=int        
        self.clipping_threshold_d = 0 # type=float  

        # Loss Options
        self.l2_loss_weight = 0 # type=float 
        self.best_k = 1 # type=int 

        # Output
        self.output_dir = os.getcwd()+ '/output/img_output'
        self.print_every = 5 # type=int
        self.checkpoint_every = 100 # type=int
        self.checkpoint_name = 'checkpoint_traffic' 
        self.checkpoint_start_from = None
        self.restore_from_checkpoint = 1 # type=int
        self.num_samples_check = 5000 # type=int        

        # Misc
        self.use_gpu = 1 # type=int
        self.timing = 0 # type=int
        self.gpu_num = "1" # type=str   


args = CreateArg() 

In [6]:
train_path=data_dir
print(train_path)

/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/test/785/


In [6]:
# data_dir="/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/train/"
# print(os.listdir(data_dir))

# from loader_image import img_loader #####
# import logging
# logger = logging.getLogger(__name__)

# for sub_dir in os.listdir(data_dir):
#     dir = data_dir+sub_dir
#     train_path = dir

#     logger.info("Initializing train dataset")

#     train_dset, train_loader = img_loader(args, train_path)

In [7]:

from loader_image import img_loader #####
import logging
logger = logging.getLogger(__name__)

import torchvision.transforms as T
# transforms = T.Compose([T.ToTensor()]) # Normalize the pixels and convert to tensor.

logger.info("Initializing train dataset")

train_dset, train_loader = img_loader(args, train_path) # train_dset은 TrajectoryDataset, train_loader는 DataLoader


In [8]:
print(train_dset)
print(train_loader)

In [9]:
print(train_dset.tensor_list)
print(type(train_dset.tensor_list))
print(len(train_dset.tensor_list))


[tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0353, 0.0235, 0.0353],
         [0.0000, 0.0000, 0.0000,  ..., 0.0392, 0.0196, 0.0118],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.7843, 0.7804, 0.7647,  ..., 0.0000, 0.0000, 0.0000],
         [0.9412, 0.9176, 0.8824,  ..., 0.0000, 0.0000, 0.0000],
         [0.7294, 0.7843, 0.8353,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0196, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.7725, 0.7686, 0.7529,  ..., 0.0000, 0.0000, 0.0000],
         [0.9294, 0.9059, 0.8706,  ..., 0.0000, 0.0000, 0.0000],
         [0.7294, 0.7843, 0.8353,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0039, 0.0000, 0.0039],
         [0.0000, 0.0000, 0.0000,  ..., 0.0353, 0.0157, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0275, 0.0275, 0

In [10]:
print(train_dset.tensor_list[0].shape)
print(train_dset.tensor_list[0])

torch.Size([3, 360, 640])
tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0353, 0.0235, 0.0353],
         [0.0000, 0.0000, 0.0000,  ..., 0.0392, 0.0196, 0.0118],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.7843, 0.7804, 0.7647,  ..., 0.0000, 0.0000, 0.0000],
         [0.9412, 0.9176, 0.8824,  ..., 0.0000, 0.0000, 0.0000],
         [0.7294, 0.7843, 0.8353,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0196, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.7725, 0.7686, 0.7529,  ..., 0.0000, 0.0000, 0.0000],
         [0.9294, 0.9059, 0.8706,  ..., 0.0000, 0.0000, 0.0000],
         [0.7294, 0.7843, 0.8353,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0039, 0.0000, 0.0039],
         [0.0000, 0.0000, 0.0000,  ..., 0.0353, 0.0157, 0.0000],
         [0.0000, 0.0000, 0.0000

In [13]:
## feature 추출을 위한 simple conv layer

In [11]:
# 2. test : 한 개의 이미지로부터 불러오기
import torch
import torch.nn as nn
class conv2DBatchNormRelu(nn.Module):
    def __init__(self, in_channels, n_filters, k_size,  stride, padding, bias=True, dilation=1):
        super(conv2DBatchNormRelu, self).__init__()

        self.cbr_unit = nn.Sequential(
            nn.Conv2d(
                int(in_channels), # 입력 채널의 수, rgb의 경우 3
                      int(n_filters), # 필터의 수
                      kernel_size=k_size, # 커널의 크기
                        padding=padding, 
                        stride=stride, 
                        bias=bias, # bias (default=True) : 편향 사용의 여부
                        dilation=dilation # 딜레이션 레이트 : default = 1
                ),
            nn.BatchNorm2d(int(n_filters)), # 2차원 이미지 데이터에 대한 배치 정규화
            nn.ReLU(inplace=True),
                                 ) 

    def forward(self, inputs):
        outputs = self.cbr_unit(inputs)
        return outputs


class SemanticImageEncoder(nn.Module):
    '''
    Tianyang:
    Simple Convolutional Encoder for semantic map images
    Input size: flexible
    Output size: 1/2 input
    '''

    def __init__(self, in_channels = 3, out_channels = 32):
        super(SemanticImageEncoder, self).__init__()

        self._encoder = nn.Sequential(
            conv2DBatchNormRelu(in_channels = in_channels, # 입력 채널 개수, RGB일 때 : 3
                                n_filters = 16, # 출력 채널 개수
                                k_size = 3,     # 컨볼루션 필터 크기 : (3,3) 
                                stride = 1,         # 필터 이동 간격 (default=1)
                                padding = 1),       # padding :(default=0) 
            
            conv2DBatchNormRelu(in_channels = 16, n_filters = 16, \
                k_size = 4,  stride = 1, padding = 2),
            
            nn.MaxPool2d((2, 2), stride=(2, 2)),

            conv2DBatchNormRelu(in_channels = 16, n_filters = out_channels, \
                k_size = 5,  stride = 1, padding = 2),
            )

    def forward(self, input): # 4차원 input (batchsize(frame 개수), 차원수, h, w)
        # encoded = self._encoder(input.type(torch.cuda.FloatTensor))
        encoded = self._encoder(input.type(torch.FloatTensor))
        return encoded

In [12]:
image_channels=3
embed_dim_image=32
image_encoder = SemanticImageEncoder(in_channels = image_channels, out_channels = embed_dim_image)

In [14]:
#image encoder 에는 4차원 input을 주어야 함 
# ValueError: expected 4D input (got 3D input)
# ft_img_0=image_encoder(train_dset.tensor_list[0])
# print(ft_img_0)

In [13]:
def list2batch(seq):
    # assemble a list of elements to batch, batch_idx: 0th dimension
    stacked = torch.tensor(seq[0]).unsqueeze(0)
    i = 1
    l = len(seq) # 리스트 길이 (이미지 파일 수)
    while i < l:
        stacked = torch.cat((stacked, torch.tensor(seq[i]).unsqueeze(0)), 0)
        i += 1
    return stacked

In [14]:
batch_img_list=list2batch(train_dset.tensor_list)
print(batch_img_list)
print(batch_img_list.shape)
print(type(batch_img_list))

/home/gpuadmin/anaconda3/envs/traffino/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/home/gpuadmin/anaconda3/envs/traffino/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0353, 0.0235, 0.0353],
          [0.0000, 0.0000, 0.0000,  ..., 0.0392, 0.0196, 0.0118],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.7843, 0.7804, 0.7647,  ..., 0.0000, 0.0000, 0.0000],
          [0.9412, 0.9176, 0.8824,  ..., 0.0000, 0.0000, 0.0000],
          [0.7294, 0.7843, 0.8353,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0196, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.7725, 0.7686, 0.7529,  ..., 0.0000, 0.0000, 0.0000],
          [0.9294, 0.9059, 0.8706,  ..., 0.0000, 0.0000, 0.0000],
          [0.7294, 0.7843, 0.8353,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0039, 0.0000, 0.0039],
          [0.0000, 0.0000, 0.0000,  ..., 0.0353, 0.0157, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0

In [15]:
ft_img_785=image_encoder(batch_img_list)
print(ft_img_785)

tensor([[[[1.7738e+00, 1.3621e+00, 0.0000e+00,  ..., 7.0177e-02,
           0.0000e+00, 1.1903e-01],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.5591e+00,
           0.0000e+00, 9.5590e-01],
          [2.1192e+00, 1.0026e+00, 1.4013e+00,  ..., 2.4221e+00,
           3.0414e+00, 1.2771e+00],
          ...,
          [2.1113e+00, 3.0786e+00, 2.4636e+00,  ..., 2.8039e+00,
           0.0000e+00, 1.8907e-01],
          [4.2761e-01, 1.1587e+00, 9.8508e-02,  ..., 1.6969e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 2.9036e-01,
           0.0000e+00, 0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 3.4037e-01,  ..., 1.1976e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 3.4083e-01, 1.1326e+00,  ..., 2.0851e+00,
           0.0000e+00, 4.3191e-01],
          ...,
          [0.0000e+00, 1.9756e+00, 3.5577e+00,  ..., 0.0000

In [16]:
print(ft_img_785.shape)
print(type(ft_img_785)) # (frame수, 32, 180, 320)
print(len(ft_img_785))

torch.Size([727, 32, 180, 320])
<class 'torch.Tensor'>
727


In [18]:
import pickle as pk

In [19]:
with open(os.path.join(data_dir, 'ft_img_785.pkl'), 'wb') as f:

    pk.dump(ft_img_785,f, protocol=4) # cannot serialize a bytes object larger than 4 GiB

In [24]:
fet_map = {}
read_path=os.path.join(data_dir, 'ft_img_785.pkl')
with open(read_path, 'rb') as handle:
    new_fet = pk.load(handle, encoding='bytes')
fet_map[read_path] = new_fet

In [30]:
print(fet_map)
# print(type(fet_map.value))

{'/home/gpuadmin/dev/Trajectory_Prediction/traffino/datasets/waterloo/img/test/785/ft_img_785.pkl': tensor([[[[1.7738e+00, 1.3621e+00, 0.0000e+00,  ..., 7.0177e-02,
           0.0000e+00, 1.1903e-01],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.5591e+00,
           0.0000e+00, 9.5590e-01],
          [2.1192e+00, 1.0026e+00, 1.4013e+00,  ..., 2.4221e+00,
           3.0414e+00, 1.2771e+00],
          ...,
          [2.1113e+00, 3.0786e+00, 2.4636e+00,  ..., 2.8039e+00,
           0.0000e+00, 1.8907e-01],
          [4.2761e-01, 1.1587e+00, 9.8508e-02,  ..., 1.6969e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 2.9036e-01,
           0.0000e+00, 0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 3.4037e-01,  ..., 1.1976e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 3.4083e-01, 1.1326e+00,  ..., 2.0851e+00,
          